# Artificial Neural Network

### Importing the libraries

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [0]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [0]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
X[:,2] = label.fit_transform(X[:,2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
print(X_train)

[[-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   1.10643166]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.41231994]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
   0.84432121]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   0.32472465]]


## Part 2 - Building the ANN

### Initializing the ANN

In [0]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [0]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # units are the number of neurons

### Adding the second hidden layer

In [0]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [0]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [0]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # if the predict y is not binary , loss = categorical_crossentropy

### Training the ANN on the Training set

In [18]:
ann.fit(X_train,y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.7517 - accuracy: 0.5236
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5371 - accuracy: 0.7958
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4843 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4608 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4440 - accuracy: 0.7977
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4286 - accuracy: 0.8080
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4128 - accuracy: 0.8167
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3994 - accuracy: 0.8219
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3880 - accuracy: 0.8303
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3779 - accura

## Part 4 - Making the predictions and evaluating the model

In [27]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

[[False]]


### Predicting the Test set results

In [31]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_pred,y_test)
print(cm)
print(accuracy_score(y_pred,y_test))

[[1510  197]
 [  85  208]]
0.859
